In [ ]:
from operator import mul
from functools import reduce
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
from pyodesys.native import native_sys
sp.init_printing()
%matplotlib inline

In [ ]:
# Fe+3 + SCN- <-> FeSCN+2
stoich_reac, stoich_prod = [(1, 1, 0), (0, 0, 1)], [(0, 0, 1), (1, 1, 0)]

In [ ]:
def reaction_rates(t, C, params):
    for i, k in enumerate(params):
        yield reduce(mul, [k] + [C[j]**n for j, n in enumerate(stoich_reac[i])])

In [ ]:
def dCdt(t, C, params):
    rates = list(reaction_rates(t, C, params))
    result = [0]*len(C)
    for r, sr, sp in zip(rates, stoich_reac, stoich_prod):
        for i in range(len(C)):
            result[i] += r*(sp[i] - sr[i])
    return result

In [ ]:
kineticsys = SymbolicSys.from_callback(dCdt, 3, 2, latex_names=['$%s$' % s for s in 'Fe^{3+} SCN^- FeSCN^{2+}'.split()],
                                   steady_state_root=True)

In [ ]:
def integrate_and_plot(odesys, plot=True, **kwargs):
    tend = 2
    result = odesys.integrate(tend, [1e-2, 2e-3, 0], [800, 8], integrator='cvode', **kwargs)
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(14, 4))
        if result.xout[-1] != tend:
            axes[0].axvline(result.xout[-1], linestyle='--', label='t = %.4f' % result.xout[-1])
        result.plot(ax=axes[0])
        result.plot(ax=axes[1], deriv=True)
        axes[1].set_yscale('symlog', linthreshy=1e-9)
        axes[1].axhline(1e-9, linestyle='--')
        axes[1].axhline(-1e-9, linestyle='--')
        for ax in axes:
            ax.set_xlim([0, tend])
    return result

In [ ]:
integrate_and_plot(kineticsys)

In [ ]:
integrate_and_plot(kineticsys, atol=1e-14, rtol=1e-14)

In [ ]:
integrate_and_plot(kineticsys, atol=1e-14, rtol=1e-14, return_on_root=True)

In [ ]:
kineticsys.roots

In [ ]:
native_override = lambda n: {
        'p_nroots': """ return 1; """,
        'p_roots': """
    const int ny = get_ny();
    std::vector<double> f(ny);
    double tot=0.0;
    rhs(x, y, &f[0]);
    for (int i=0; i<ny; ++i){
        tot += std::min(std::abs(f[i]/m_atol[i]), std::abs(f[i]/y[i]/m_rtol));
    }
    out[0] = tot/ny - %d;
    this->nrev++;
    return AnyODE::Status::success;
""" % n}
nativesys = native_sys['cvode'].from_other(kineticsys, namespace_override=native_override(1000))
print(open(next(filter(lambda s: s.endswith('.cpp'), nativesys._native._written_files))).read())

In [ ]:
res = integrate_and_plot(nativesys, atol=1e-14, rtol=1e-14, return_on_root=True)
print(res.info['success'])

In [ ]:
integrate_and_plot(nativesys, atol=1e-13, rtol=1e-13, return_on_root=True)

In [ ]:
integrate_and_plot(nativesys, atol=1e-12, rtol=1e-12, return_on_root=True)

In [ ]:
integrate_and_plot(nativesys, atol=1e-11, rtol=1e-11, return_on_root=True)

In [ ]:
integrate_and_plot(nativesys, atol=1e-10, rtol=1e-10, return_on_root=True)

In [ ]:
def plot_tss_conv(factor, tols, ax):
    native = native_sys['cvode'].from_other(kineticsys, namespace_override=native_override(factor))
    tol_kw = dict(plot=False, return_on_root=True, nsteps=2000)
    tss = [integrate_and_plot(native, atol=tol, rtol=tol, **tol_kw).xout[-1] for tol in tols]
    ax.semilogx(tols, tss, label=factor)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
tols = np.logspace(-15, -10, 200)
for factor in [1e2, 1e3, 1e4, 1.1e4, 1e5, 1e6, 1e7]:
    plot_tss_conv(factor, tols, ax)
ax.legend()